In [88]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder,boxscoretraditionalv2
from nba_api.stats.endpoints import leaguedashplayerstats,leaguegamelog
print('Set up completed!')

Set up completed!


In [78]:
finder = leaguegamefinder.LeagueGameFinder(season_nullable="2023-24",league_id_nullable="00",season_type_nullable="Regular Season")
games = finder.get_data_frames()[0]
games.head()



,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612755,PHI,Philadelphia 76ers,0022301192,2024-04-14,PHI vs. BKN,W,240,107,...,0.733,13,44,57,30,8,6,12,16,21.0
1,22023,1610612759,SAS,San Antonio Spurs,0022301197,2024-04-14,SAS vs. DET,W,241,123,...,0.714,12,42,54,33,9,3,13,17,28.0
2,22023,1610612752,NYK,New York Knicks,0022301190,2024-04-14,NYK vs. CHI,W,265,120,...,0.762,16,37,53,27,7,6,21,17,1.0
3,22023,1610612756,PHX,Phoenix Suns,0022301194,2024-04-14,PHX @ MIN,W,244,125,...,0.833,13,19,32,28,13,3,12,24,19.0
4,22023,1610612746,LAC,LA Clippers,0022301199,2024-04-14,LAC vs. HOU,L,240,105,...,0.882,16,35,51,23,13,8,12,14,-11.0


In [91]:
players_stats = leaguedashplayerstats.LeagueDashPlayerStats(season="2023-24").get_data_frames()[0]
stars = players_stats.sort_values('PTS', ascending=False).drop_duplicates('TEAM_ID')
stars_list = stars[['TEAM_ID', 'PLAYER_ID','PLAYER_NAME']]

player_logs = leaguegamelog.LeagueGameLog(
    season='2023-24', 
    player_or_team_abbreviation='P'
).get_data_frames()[0]

star_attendance = player_logs[player_logs['PLAYER_ID'].isin(stars_list['PLAYER_ID'])][['GAME_ID', 'PLAYER_ID']]
star_attendance['IS_ACTIVE'] = 1
print(stars_list.head())
print(star_attendance.head())

        TEAM_ID  PLAYER_ID              PLAYER_NAME
372  1610612742    1629029              Luka Dončić
501  1610612760    1628983  Shai Gilgeous-Alexander
184  1610612749     203507    Giannis Antetokounmpo
233  1610612752    1628973            Jalen Brunson
434  1610612743     203999             Nikola Jokić
       GAME_ID  PLAYER_ID  IS_ACTIVE
2   0022300061     203999          1
24  0022300062     201939          1
27  0022300061     203076          1
29  0022300062     201142          1
47  0022300067    1628378          1


In [85]:
games = games.sort_values(by=['TEAM_ID', 'GAME_DATE'])
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
cols_to_roll = ['PTS', 'FG_PCT', 'AST', 'REB', 'TOV']
for col in cols_to_roll:
    games['ROLL_' + col] = games.groupby('TEAM_ID')[col].transform(lambda x: x.rolling(window=10,min_periods=1).mean().shift(1))
games['CUM_WINS'] = games.groupby('TEAM_ID')['WL'].transform(lambda x: (x == 'W').cumsum().shift(1).fillna(0))
games['GAMES_PLAYED'] = games.groupby('TEAM_ID').cumcount()
games['SEASON_WIN_PCT'] = games['CUM_WINS'] / games['GAMES_PLAYED']
games['SEASON_WIN_PCT'] = games['SEASON_WIN_PCT'].fillna(0)

games['DAYS_REST'] = games.groupby('TEAM_ID')['GAME_DATE'].diff().dt.days
games['DAYS_REST'] = games['DAYS_REST'].clip(upper=4).fillna(4)
games = games.dropna()
print(games[:11])
games.shape

     SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
2420     22023  1610612737               ATL  Atlanta Hawks  0022300079   
2389     22023  1610612737               ATL  Atlanta Hawks  0022300097   
2376     22023  1610612737               ATL  Atlanta Hawks  0022300104   
2350     22023  1610612737               ATL  Atlanta Hawks  0022300117   
2295     22023  1610612737               ATL  Atlanta Hawks  0022300135   
2257     22023  1610612737               ATL  Atlanta Hawks  0022300155   
2222     22023  1610612737               ATL  Atlanta Hawks  0022300172   
2198     22023  1610612737               ATL  Atlanta Hawks  0022300175   
2153     22023  1610612737               ATL  Atlanta Hawks  0022300018   
2132     22023  1610612737               ATL  Atlanta Hawks  0022300193   
2123     22023  1610612737               ATL  Atlanta Hawks  0022300029   

      GAME_DATE      MATCHUP WL  MIN  PTS  ...  PLUS_MINUS  GAMES_PLAYED  \
2420 2023-10-27  ATL vs

(2429, 37)

In [144]:
stats_cols = ['GAME_ID','TEAM_NAME','TEAM_ID','WL','SEASON_WIN_PCT','DAYS_REST','GAMES_PLAYED','GAME_DATE'] + [col for col in games.columns if 'ROLL_' in col]


home_games = games[games['MATCHUP'].str.contains('vs')][stats_cols].copy()
away_games = games[games['MATCHUP'].str.contains(' @ ')][stats_cols].copy()

home_games = home_games.add_prefix('HOME_').rename(columns={'HOME_GAME_ID': 'GAME_ID','HOME_GAME_DATE': 'GAME_DATE'})
away_games = away_games.add_prefix('AWAY_').rename(columns={'AWAY_GAME_ID': 'GAME_ID'})

model_df = pd.merge(home_games, away_games, on='GAME_ID')
model_df['HOME_WIN'] = (model_df['HOME_WL'] == 'W').astype(int)



#games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
#games = games.sort_values(['TEAM_ID','GAME_DATE'])



model_df.head()


,GAME_ID,HOME_TEAM_NAME,HOME_TEAM_ID,HOME_WL,HOME_SEASON_WIN_PCT,HOME_DAYS_REST,HOME_GAMES_PLAYED,GAME_DATE,HOME_ROLL_PTS,HOME_ROLL_FG_PCT,...,AWAY_SEASON_WIN_PCT,AWAY_DAYS_REST,AWAY_GAMES_PLAYED,AWAY_GAME_DATE,AWAY_ROLL_PTS,AWAY_ROLL_FG_PCT,AWAY_ROLL_AST,AWAY_ROLL_REB,AWAY_ROLL_TOV,HOME_WIN
0,0022300079,Atlanta Hawks,1610612737,L,0.000000,2.0,1,2023-10-27,110.00,0.41900,...,0.000000,2.0,1,2023-10-27,104.000000,0.371000,24.00,47.000000,9.000000,0
1,0022300104,Atlanta Hawks,1610612737,W,0.333333,1.0,3,2023-10-30,119.00,0.46900,...,0.500000,2.0,2,2023-10-30,100.000000,0.426000,25.00,54.500000,14.000000,1
2,0022300117,Atlanta Hawks,1610612737,W,0.500000,2.0,4,2023-11-01,121.00,0.49125,...,0.333333,2.0,3,2023-11-01,113.333333,0.445333,26.00,43.333333,13.333333,1
3,0022300175,Atlanta Hawks,1610612737,L,0.625000,2.0,8,2023-11-11,121.75,0.47550,...,0.500000,3.0,8,2023-11-11,107.500000,0.457250,25.75,43.250000,14.750000,0
4,0022300193,Atlanta Hawks,1610612737,L,0.600000,1.0,10,2023-11-15,120.90,0.47580,...,0.500000,2.0,10,2023-11-15,108.400000,0.428600,22.50,47.700000,11.900000,0


In [146]:
star_map = dict(zip(stars_list['TEAM_ID'], stars_list['PLAYER_ID']))
model_df['HOME_STAR_ID'] = model_df['HOME_TEAM_ID'].map(star_map)
model_df['AWAY_STAR_ID'] = model_df['AWAY_TEAM_ID'].map(star_map)

# 1. Create the set for instant lookup (same as before)
active_pairs = set(zip(star_attendance['GAME_ID'], star_attendance['PLAYER_ID']))

In [147]:
def get_motivation_level(win_pct, games_played):
    # Motivation only becomes a major factor in the final 20 games
    if games_played > 62: 
        # TANKING: Bottom dwellers (e.g., win pct < 35%)
        if win_pct < 0.35: return 1 
        # RESTING: Top seeds who have clinched (e.g., win pct > 72%)
        if win_pct > 0.72: return 1
        # URGENT: The "Play-in" hunt (e.g., 40% to 55% win pct)
        if 0.40 <= win_pct <= 0.55: return 3
    
    return 2 # Default/Normal Motivation

# Apply to Home and Away
model_df['HOME_MOTIVATION'] = model_df.apply(lambda x: get_motivation_level(x['HOME_SEASON_WIN_PCT'], x['HOME_GAMES_PLAYED']), axis=1)
model_df['AWAY_MOTIVATION'] = model_df.apply(lambda x: get_motivation_level(x['AWAY_SEASON_WIN_PCT'], x['AWAY_GAMES_PLAYED']), axis=1)

# THE FEATURE: The Motivation Gap
# Positive = Home team is more "desperate" to win
# Negative = Away team is more "desperate"
model_df['DIFF_MOTIVATION'] = model_df['HOME_MOTIVATION'] - model_df['AWAY_MOTIVATION']
print(model_df[:20])

       GAME_ID HOME_TEAM_NAME  HOME_TEAM_ID HOME_WL  HOME_SEASON_WIN_PCT  \
0   0022300079  Atlanta Hawks    1610612737       L             0.000000   
1   0022300104  Atlanta Hawks    1610612737       W             0.333333   
2   0022300117  Atlanta Hawks    1610612737       W             0.500000   
3   0022300175  Atlanta Hawks    1610612737       L             0.625000   
4   0022300193  Atlanta Hawks    1610612737       L             0.600000   
5   0022300029  Atlanta Hawks    1610612737       L             0.545455   
6   0022300039  Atlanta Hawks    1610612737       L             0.500000   
7   0022300227  Atlanta Hawks    1610612737       W             0.461538   
8   0022301209  Atlanta Hawks    1610612737       L             0.473684   
9   0022300296  Atlanta Hawks    1610612737       L             0.428571   
10  0022300352  Atlanta Hawks    1610612737       W             0.400000   
11  0022300393  Atlanta Hawks    1610612737       L             0.428571   
12  00223004

In [153]:
for col in ['PTS', 'FG_PCT', 'AST', 'REB', 'TOV']:
    model_df[f'DIFF_{col}'] = model_df[f'HOME_ROLL_{col}'] - model_df[f'AWAY_ROLL_{col}']
model_df['DIFF_SEASON_WIN_PCT'] = model_df['HOME_SEASON_WIN_PCT'] - model_df['AWAY_SEASON_WIN_PCT']
model_df['DIFF_REST'] = model_df['HOME_DAYS_REST'] - model_df['AWAY_DAYS_REST']
model_df['DIFF_STAR_ACTIVE'] = [
    (1 if (game, h_star) in active_pairs else 0) - (1 if (game, a_star) in active_pairs else 0)
    for game, h_star, a_star in zip(model_df['GAME_ID'], model_df['HOME_STAR_ID'], model_df['AWAY_STAR_ID'])
]

diff_features = [col for col in model_df.columns if 'DIFF_' in col]

model_df['GAME_DATE'] = pd.to_datetime(model_df['GAME_DATE'])
model_df = model_df.sort_values('GAME_DATE').reset_index(drop=True)

X = model_df[diff_features]
y = model_df['HOME_WIN']


split_index = int(len(model_df)*0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

print(f'Training on {len(X_train)} games, testing on {len(X_test)} games')
print(model_df[:20])

Training on 971 games, testing on 243 games
       GAME_ID          HOME_TEAM_NAME  HOME_TEAM_ID HOME_WL  \
0   0022300076      Los Angeles Lakers    1610612747       W   
1   0022300079           Atlanta Hawks    1610612737       L   
2   0022300084        Dallas Mavericks    1610612742       W   
3   0022300081     Cleveland Cavaliers    1610612739       L   
4   0022300077       Charlotte Hornets    1610612766       L   
5   0022300086  Portland Trail Blazers    1610612757       L   
6   0022300080          Boston Celtics    1610612738       W   
7   0022300087        Sacramento Kings    1610612758       L   
8   0022300083       San Antonio Spurs    1610612759       W   
9   0022300085               Utah Jazz    1610612762       W   
10  0022300078       Memphis Grizzlies    1610612763       L   
11  0022300082           Chicago Bulls    1610612741       W   
12  0022300088         Detroit Pistons    1610612765       W   
13  0022300092         Toronto Raptors    1610612761       L

In [154]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = model = XGBClassifier(
    n_estimators=100, 
    max_depth=3,    
    learning_rate=0.1, 
    random_state=42
)
model.fit(X_train, y_train)

preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6214


In [155]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

# Calculate the matrix
cm = confusion_matrix(y_test, preds)

# Format for Plotly
z = cm[::-1] # Flip for heatmap display
x = ['Predicted Loss', 'Predicted Win']
y = ['Actual Win', 'Actual Loss']

# Create the heatmap
fig = ff.create_annotated_heatmap(z, x=x, y=y, colorscale='Viridis')
fig.update_layout(title_text='<b>Confusion Matrix: Where is the model failing?</b>')
fig.show()

In [156]:
import plotly.express as px

# Get feature importance
importance = model.feature_importances_
feature_names = X.columns

# Create a DataFrame for plotting
fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance}).sort_values(by='Importance', ascending=True)

# Plot with Plotly
fig = px.bar(fi_df, x='Importance', y='Feature', orientation='h',
             title='<b>Which Feature is the "MVP" of the Model?</b>',
             template='plotly_dark')
fig.show()

In [164]:
# 1. Filter for the "Golden Window" (Games 50 to 60)
# This ensures we test on stable, high-motivation games
test_subset = model_df[(model_df['HOME_GAMES_PLAYED'] >= 30) & 
                       (model_df['HOME_GAMES_PLAYED'] <= 70)].copy()

# 2. Define the features (Ensure this matches your final training list)
final_features = [col for col in model_df.columns if col.startswith('DIFF_')]

X_final_test = test_subset[final_features]
y_final_actual = test_subset['HOME_WIN']

# 3. Get Predictions and Probabilities
final_preds = model.predict(X_final_test)
final_probs = model.predict_proba(X_final_test)[:, 1]

# 4. Create the Results Table
test_results = pd.DataFrame({
    'Home': test_subset['HOME_TEAM_NAME'],
    'Away': test_subset['AWAY_TEAM_NAME'],
    'Home_GP': test_subset['HOME_GAMES_PLAYED'],
    'Win_Prob': [f"{p:.1%}" for p in final_probs],
    'Predicted': final_preds,
    'Actual': y_final_actual.values
})

# 5. Calculate Accuracy for this specific window
accuracy = (test_results['Predicted'] == test_results['Actual']).mean() * 100

print(f"--- GOLDEN WINDOW RESULTS (Games 30-70) ---")
print(f"Accuracy: {accuracy:.1f}%")
print(test_results[30:70])

--- GOLDEN WINDOW RESULTS (Games 30-70) ---
Accuracy: 76.9%
                       Home                    Away  Home_GP Win_Prob  \
462       Memphis Grizzlies        Sacramento Kings       31    41.0%   
463   Oklahoma City Thunder           Brooklyn Nets       30    78.8%   
464    New Orleans Pelicans      Los Angeles Lakers       32    73.6%   
465             LA Clippers              Miami Heat       31    73.3%   
466          Denver Nuggets       Charlotte Hornets       34    89.1%   
467         Milwaukee Bucks          Indiana Pacers       32    62.2%   
468            Phoenix Suns  Portland Trail Blazers       32    69.1%   
469         Toronto Raptors     Cleveland Cavaliers       32    46.4%   
470               Utah Jazz        Dallas Mavericks       33    43.8%   
471         Houston Rockets         Detroit Pistons       30    87.1%   
472         New York Knicks  Minnesota Timberwolves       32    65.3%   
473    New Orleans Pelicans           Brooklyn Nets       33    